## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [2]:
from gensim.models import Word2Vec, KeyedVectors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import judicial_splitter
import collections
from pymystem3 import Mystem
mystem = Mystem()
import numpy as np
from itertools import groupby
from tqdm import tqdm_notebook as tqdm
import os
import pandas as pd
from gensim.test.utils import get_tmpfile
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import operator
import re
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
import math
from model import Model
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
from operator import itemgetter 

# Подготовка

In [528]:
main_dir = '/Users/irene/Downloads/avito_dogs'
files_list = []
for root, dirs, files in os.walk(main_dir):
    for name in files:
        if not '.DS_Store' in name:
            files_list.append(os.path.join(root, name))

In [5]:
def read_parse(path):
    with open(path, 'r') as f:  
        info = f.readlines()
    title = info[0].replace('Название: ', '').strip()
    num_date = info[1].replace('Номер и дата объявления: ', '').strip()
    author = info[2].replace('Информация об авторе: ', '').strip()
    address = info[3].replace('Адрес собаки: ', '').strip().strip()
    breed = info[4].replace('Порода: ', '').strip()
    price = info[5].replace('Цена: ', '').strip()
    description = info[6].replace('Описание собаки: ', '').strip()
    url = info[7].replace('URL: ', '').strip()
    return title, num_date, author, address, breed, price, description, url

In [ ]:
read_parse(files_list[0])

In [ ]:
df = pd.DataFrame(columns=['title', 'num_date', 'author', 'address', 'breed', 'price', 'description', 'url'])
for i, path in tqdm(enumerate(files_list)):
    title, num_date, author, address, breed, price, description, url = read_parse(path)
    df.loc[i] = [title, num_date, author, address, breed, price, description, url]

In [ ]:
df.head()

In [ ]:
df.to_csv('avito_df.csv')

In [3]:
data = pd.read_csv('avito_df.csv')

In [4]:
data.head()

,Unnamed: 0,title,num_date,author,address,breed,price,description,url
0,0,Гончая,"No 1710785348, размещено 3 октября в 17:35","Юлия, Частное лицо, На Авито c сентября 2015, ...","Абакан,",лабрадор,100 ₽,"Найден Калинино 3, пес очень умный, знает ко...",https://www.avito.ru/abakan/sobaki/gonchaya_17...
1,1,Пекинес,"No 925865570, размещено вчера в 07:46","Анна, Частное лицо, На Авито c марта 2014, Зав...","Абакан,",пекинес,10 000 ₽,Щенки Пекинеса 2 кобеля. Дата рождения 11 мая ...,https://www.avito.ru/abakan/sobaki/pekines_925...
2,2,"Померанский Шпиц бело крем, белые","No 824574165, размещено 3 октября в 22:54","алексей, На Авито c января 2016, Завершено 52...","Абакан,",шпиц,26 000 ₽,Продаются щенки померанского шпица. бело кремо...,https://www.avito.ru/abakan/sobaki/pomeranskiy...
3,3,Продажа собаки,"No 565618762, размещено 3 октября в 20:48","Наталья, Частное лицо, На Авито c апреля 2013,...","Абакан,",другая,Договорная,"ждут хорошего хозяина, собака умная, чуткая, с...",https://www.avito.ru/abakan/sobaki/prodazha_so...
4,4,Продажа собаки,"No 596714096, размещено 3 октября в 20:45","Наталья, Частное лицо, На Авито c апреля 2013,...","Абакан,",другая,Договорная,украшение для вашей семьи,https://www.avito.ru/abakan/sobaki/prodazha_so...


In [5]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    if del_digit:
        input_text = re.sub('[0-9]', '', input_text)
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

# TF.IDF

In [6]:
def compute_tf(text):
    tf_text = collections.Counter(text)
    for i in tf_text:
        tf_text[i] = tf_text[i]/float(len(text))
    return tf_text

In [7]:
def compute_idf(word, corpus):
    return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

In [8]:
def compute_tfidf(corpus):
    documents_list = []
    for text in tqdm(corpus):
        tf_idf_dictionary = {}
        computed_tf = compute_tf(text)
        for word in computed_tf:
            tf_idf_dictionary[word] = computed_tf[word] * compute_idf(word, corpus)
        documents_list.append(tf_idf_dictionary)
    return documents_list

#corpus = [['pasta', 'la', 'vista', 'baby', 'la', 'vista'], 
#['hasta', 'siempre', 'comandante', 'baby', 'la', 'siempre'], 
#['siempre', 'comandante', 'baby', 'la', 'siempre']]
#compute_tfidf(corpus)

In [9]:
# description
# num_date, price, title, address, breed, author  - доп признаки (отдельно обрабатываются)
corpus = [preprocessing(d) for d in data['description']]
tfidf = compute_tfidf(corpus)

A Jupyter Widget

In [108]:
def compute_new_tfidf(word, quary, corpus):
    try:
        quary = preprocessing(quary)
        computed_tf = compute_tf(quary)[word]
        tfidf = computed_tf * compute_idf(word, corpus)
    except:
        tfidf = 0.0
    return tfidf

# w2v

In [10]:
# если модель без тэгов
model_without_pos = Word2Vec.load('/Users/irene/Downloads/IR/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')

In [110]:
def get_w2v_vectors_paragraph(paragraph, model, tfidf, ind, multiply_tfidf=True, pos=False):
    """Получает вектор для параграфа"""
    lemmas_paragraph = preprocessing(paragraph)
    #print('lemmas_paragraph', lemmas_paragraph)
    if len(lemmas_paragraph) == 0:
        return np.zeros(300)
    else:
        vector_paragraph = []
        for lemma in lemmas_paragraph:
            if pos:
                lemma = lemma + '_' + get_pos(lemma)
            try:
                if multiply_tfidf:
                    tfidf = compute_new_tfidf(lemma, paragraph, corpus)
                    #print(lemma, tfidf)
                    vector = model.wv[lemma] * tfidf
                else:
                    vector = model.wv[lemma]
            except:
                vector = np.zeros(300)
            vector_paragraph.append(vector)
        vec = np.array(vector_paragraph).sum(axis=0) / len(vector_paragraph)
        return vec.tolist()

In [111]:
#get_w2v_vectors_paragraph('коричневый бульдог', model_without_pos, 'dfn', 'djvsd', multiply_tfidf=True, pos=False)

In [12]:
def get_w2v_vectors_text(text, model, tfidf, ind, len_par=4, multiply_tfidf=True, pos=False):
    """Получает массив векторов параграфов"""
    paragraphs = judicial_splitter.splitter(text, len_par)
    return [(paragraph, get_w2v_vectors_paragraph(paragraph, model, tfidf, ind, multiply_tfidf=multiply_tfidf, pos=pos)) for paragraph in paragraphs]

In [13]:
def save_w2v_base(answers, model, tfidf, len_par=4, multiply_tfidf=True, pos=False):
    """Индексирует всю базу для поиска через word2vec"""
    id_answer = []
    text_of_paragraph = []
    w2v = []
    for i, answer in tqdm(enumerate(answers)):
        v_paragraphs = get_w2v_vectors_text(answer, model, tfidf, i, len_par=len_par, multiply_tfidf=multiply_tfidf, pos=pos)
        for v_p in v_paragraphs:
            id_answer.append(i)
            text_of_paragraph.append(v_p[0])
            w2v.append(v_p[1])
    return id_answer, text_of_paragraph, w2v

In [14]:
def create_df(name_cols, data_cols):
    df = {}
    for i, name in enumerate(name_cols):
        df[name] = data_cols[i]
    df = pd.DataFrame(data=df)
    return df

In [15]:
id_answer, text_of_paragraph, w2v = save_w2v_base(data['description'], model_without_pos, tfidf, len_par=2, multiply_tfidf=False)

A Jupyter Widget

In [16]:
df = create_df(['id_answer', 'text_of_paragraph', 'w2v'], [id_answer, text_of_paragraph, w2v])

# С TF.IDF

In [17]:
id_answer_tfidf, text_of_paragraph_tfidf, w2v_tfidf = save_w2v_base(data['description'], model_without_pos, tfidf, len_par=2, multiply_tfidf=True)
df['w2v_tfidf'] = w2v_tfidf

A Jupyter Widget

# d2v

In [18]:
def train_doc2vec(data, epochs=100):
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, 
                min_alpha=0.025, epochs=epochs, workers=4, dm=1)

    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    return model

In [19]:
fname = get_tmpfile("model_doc2vec_avito")
model_doc2vec = train_doc2vec(df['text_of_paragraph'], epochs=1000)
model_doc2vec.save(fname)
model_doc2vec = Doc2Vec.load(fname)

In [20]:
def get_d2v_vectors(paragraph, model_doc2vec, steps=5, alpha=0.1):
    """Получает вектор параграфа"""
    lemmas_paragraph = preprocessing(paragraph, del_stopwords=False)
    model_doc2vec.random.seed(100)
    vector = model_doc2vec.infer_vector(lemmas_paragraph, steps=steps, alpha=alpha)
    return vector.tolist()

def save_d2v_base(paragraphs, model_doc2vec, steps=5, alpha=0.1):
    """Индексирует всю базу для поиска через doc2vec"""
    vectors_d2v = []
    for par in tqdm(paragraphs):
        vectors_d2v.append(get_d2v_vectors(par, model_doc2vec, steps=steps, alpha=alpha))
    return vectors_d2v

In [21]:
df['d2v_hypo'] = save_d2v_base(df['text_of_paragraph'], model_doc2vec, steps=10, alpha=0.025)

A Jupyter Widget

In [22]:
df.head()

,id_answer,text_of_paragraph,w2v,w2v_tfidf,d2v_hypo
0,0,"Найден Калинино 3, пес очень умный, знает ко...","[0.0011089619947597384, -0.011527637019753456,...","[-0.0002580559521447867, -0.000550129043404012...","[0.5840355753898621, -0.32408908009529114, -0...."
1,1,Щенки Пекинеса 2 кобеля. Дата рождения 11 мая ...,"[0.03224942533092366, 0.03836751843078269, 0.0...","[0.0009409348325182995, 0.0009336340889502834,...","[1.439154028892517, 0.5374758839607239, 0.6894..."
2,1,Воспитанные.росли в частном доме. На улицу про...,"[-0.01494167186319828, 0.017454788088798523, 0...","[-0.0006672082236036658, 0.0011105554876849055...","[0.4032999873161316, -0.22038237750530243, 0.0..."
3,1,Питание смешанное. Очень привязанные к друг др...,"[-0.007580178324133158, 0.001755695790052414, ...","[-0.0001709196512820199, 8.916016668081284e-06...","[0.07304802536964417, 0.14767879247665405, 0.3..."
4,1,Поэтому желательно в одну семью. Прежде чем вз...,"[0.009978953748941422, 0.0028232107870280743, ...","[0.0005157773848623037, 0.0003232476010452956,...","[0.8697100281715393, 0.20530551671981812, 0.06..."


In [23]:
df.to_csv('avito_df_vecors.csv')

# w2v + d2v (общие функции для поиска)

In [24]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

In [25]:
def res_v(vectors, names_doc, v_quary):
    res = []
    for i, vector in enumerate(vectors):
        cos_sim = similarity(v_quary, vector)
        res.append([names_doc[i], cos_sim, i])
    res.sort(key=operator.itemgetter(1), reverse=True)
    return res

In [26]:
def res_without_dupl(res, top=5):
    res_without_dupl = set()
    inds = []
    for ind, r in enumerate(res):
        if r[0] in res_without_dupl:
            continue
        else:
            if len(res_without_dupl) == top:
                break
            res_without_dupl.add(r[0])
            inds.append(ind)
        ind += 1
    return itemgetter(*inds)(res)

In [27]:
res = [[1, 0.9, 2], [1, 0.8, 1], [2, 0.7, 1], [2, 0.5, 3], [2, 0.5, 2], [3, 0.3, 1], [3, 0.2, 2], [4, 0.2, 1], [5, 0.2, 1], [6, 0.1, 1]]

In [28]:
res_without_dupl(res, top=1)

[1, 0.9, 2]

In [116]:
def search_w2v(quary, model, vectors_w2v, names_doc, tfidf, ind, multiply_tfidf=True, pos=False, top=5):
    v_quary = get_w2v_vectors_paragraph(quary, model, tfidf, ind, multiply_tfidf=multiply_tfidf, pos=pos)
    #print(v_quary)
    res = res_v(vectors_w2v, names_doc, v_quary)
    res = res_without_dupl(res, top=top)
    return res

def search_d2v(quary, model, vectors_d2v, names_doc, steps=5, alpha=0.1, top=5):
    v_quary = get_d2v_vectors(quary, model, steps=steps, alpha=alpha)
    res = res_v(vectors_d2v, names_doc, v_quary)
    res = res_without_dupl(res, top=top)
    return res

In [117]:
search_w2v(query, model_without_pos, df['w2v_tfidf'], df['id_answer'], 'sdfa', 'dfs', multiply_tfidf=True, pos=False, top=5)

([4365, 0.8449181136334749, 8815],
 [2528, 0.8337310792077939, 4962],
 [2128, 0.8176431519677047, 4039],
 [2345, 0.8081612212255139, 4555],
 [4675, 0.782490373118999, 9356])

# Обратный индекс

In [30]:
lemmatized_texts = []
for each_f in tqdm(data['description']):
    lemmatized = ' '.join([x for x in preprocessing(each_f) if x != ' '])
    lemmatized_texts.append(lemmatized)

A Jupyter Widget

In [31]:
vec = CountVectorizer()
X = vec.fit_transform(lemmatized_texts)
df_index = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
words = list(vec.get_feature_names())

In [32]:
df_index.head()

,aa,abeliya,abigail,abkc,above,absoluteli,absolutely,acti,adamant,adba,...,ярильд,яркий,яркия,ярко,ярославль,ярун,ярцево,ярый,ясно,яша
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
df_index.shape

(5691, 12365)

In [34]:
def inverted_index(df) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    files = []
    for word in df:
        sub = []
        docs = np.where(df[word] > 0)[0]
        for f in docs:
            dl = len(lemmatized_texts[f].split())
            fr = round(df[word][f]/dl, 4)
            sub.append([f, dl, fr])
        files.append(sub)
    index = pd.DataFrame(data={'Слово': words, 'Информация': files})
    return index

In [35]:
index = inverted_index(df_index)

In [36]:
index.head()

,Информация,Слово
0,"[[1298, 88, 0.0114], [3445, 45, 0.0222]]",aa
1,"[[1940, 103, 0.0097]]",abeliya
2,"[[158, 110, 0.0182]]",abigail
3,"[[2073, 64, 0.0156]]",abkc
4,"[[4791, 36, 0.0278]]",above


In [37]:
from math import log

k1 = 2.0
b = 0.75
avgdl = round(sum([len(q.split(' ')) for q in lemmatized_texts])/len(lemmatized_texts))#средняя длина док-ов в коллекции
N = len(lemmatized_texts)

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = math.log((N-n+0.5)/(n+0.5)) * (k1+1)*qf/(qf+k1*(1-b+b*(dl/avgdl)))
    return score

In [38]:
def compute_sim(lemma, inverted_index) -> float:
    """
    Compute similarity score between word in search query and all document  from collection
    :return: score
    """
    doc_list = list(inverted_index.loc[inverted_index['Слово'] == lemma]['Информация'])[0]
    #print(len(doc_list))
    relevance_dict = {}
    for doc in doc_list:
        relevance_dict[doc[0]] = score_BM25(doc[2], doc[1], avgdl, k1, b, N, len(doc_list))
    return relevance_dict

In [83]:
def get_search_result(query, top=5) -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    query = [que for que in preprocessing(query) if que in words]
    #print(query)
    res = {}
    for word in query:
        relevance_dict = compute_sim(word, index)
        res = {k: res.get(k, 0) + relevance_dict.get(k, 0) for k in set(res) | set(relevance_dict)}
    return sorted(res.items(), key=operator.itemgetter(1), reverse=True)[0:top]

In [40]:
len(get_search_result('бульдог', top=1384))

85

# Blending

## * w2v + d2v

In [41]:
def blend_d2v_w2v(res_w2v, res_d2v, v, top=5):
    res_w2v = sorted(res_w2v, key = lambda x: (x[0], x[2]))
    res_d2v = sorted(res_d2v, key = lambda x: (x[0], x[2]))
    ranges = []
    for i, res3 in enumerate(res_w2v):
        new_range = res3[1] * v + res_d2v[i][1] * (1-v)
        ranges.append((res3[0], new_range))
    return sorted(ranges, key = lambda x: (x[1]), reverse=True)[0:top]

In [42]:
res_w2v = [[2, 0.3 , 1], [2, 0.2, 2], [2, 0.1, 3], [1, 0.1, 1], [1, 0.1, 2]]
res_d2v = [[2, 0.8, 2], [2, 0.7, 1], [1, 0.5, 1], [2, 0.2, 3], [1, 0.1, 2]]

In [43]:
blend_d2v_w2v(res_w2v, res_d2v, 0.8, top=5)

[(2, 0.37999999999999995),
 (2, 0.32),
 (1, 0.18),
 (2, 0.12000000000000001),
 (1, 0.1)]

## * w2v + inverted index

In [44]:
def norm(vector):
    a = np.asarray(vector)
    return np.interp(a, (a.min(), a.max()), (-1, +1))

In [153]:
def f(x):
    m = x[1] / x[0]
    print(m)
    return m

In [185]:
def mean_w2v(res_w2v):
    df = pd.DataFrame(list(res_w2v))
    df = df.groupby(0)[1].agg(["count", "sum", "mean"])
    m = df['mean']
    return [(i, el) for i, el in enumerate(m)]

In [186]:
f = [[87, 0.7786801283473814, 168], [5004, 0.7450444104011303, 9957], [3057, 0.7438936781176754, 6184], [2333, 0.7431848368026404, 4536], [1921, 0.7246510872323191, 3610], [949, 0.7221993874777424, 1807]]

In [187]:
mean_w2v(f)

[(0, 0.7786801283473814),
 (1, 0.7221993874777424),
 (2, 0.7246510872323191),
 (3, 0.7431848368026404),
 (4, 0.7438936781176754),
 (5, 0.7450444104011303)]

In [197]:
def blend_w2v_index(res_w2v, res_index, v, top=5):
    res_w2v = mean_w2v(res_w2v)
    #print(res_w2v)
    res_w2v = sorted(res_w2v, key = lambda x: (x[0]))
    res_index = sorted(res_index, key = lambda x: (x[0]))
    files_ind = [r[0] for r in res_index]
    res_w2v = np.asarray(res_w2v)[files_ind]
    res_w2v_norm = [(i, j) for i, j in enumerate(norm([l[1] for l in res_w2v]))]
    res_index_norm = [(i, j) for i, j in enumerate(norm([d[1] for d in res_index]))]
    ranges = []
    for i, res3 in enumerate(res_w2v_norm):
        new_range = res3[1] * v + res_index_norm[i][1] * (1-v)
        ranges.append((res3[0], new_range))
    return sorted(ranges, key = lambda x: (x[1]), reverse=True)[0:top]

In [50]:
res_w2v = [[0, 0.5, 3], [0, 0.5, 2], [1, 0.3, 1], [1, 0.2, 2], [2, 0.2, 1], [3, 0.2, 1], [4, 0.1, 1], [0, 0.1, 1], [54, 0.1, 1]]
res_index = [(1, 65), (0, 6), (2, 5), (4, 2), (3, 3)]

In [52]:
blend_w2v_index(res_w2v, res_index, 0.5, top=5)

[(1, 0.5624999999999999),
 (0, 0.06349206349206349),
 (2, -0.5773809523809524),
 (3, -0.6091269841269842),
 (4, -1.0)]

In [205]:
def search(search_method, query, model_without_pos, model_doc2vec, vectors_w2v, vectors_d2v, names_doc, tfidf, ind_text, num_par_coll, num_text_coll, top=5):
    try:
        if search_method == 'inverted_index':
            search_result = get_search_result(query, top=top)
        elif search_method == 'word2vec':
            search_result = search_w2v(query, model_without_pos, vectors_w2v, names_doc, tfidf, ind_text, multiply_tfidf=True, pos=False, top=top)
        elif search_method == 'doc2vec':
            search_result = search_d2v(query, model_doc2vec, vectors_d2v, names_doc, steps=5, alpha=0.1, top=top)
        elif search_method == 'doc2vec+word2vec':
            top_w2v = search_w2v(query, model_without_pos, vectors_w2v, names_doc, tfidf, ind_text, multiply_tfidf=True, pos=False, top=num_par_coll)
            top_d2v = search_d2v(query, model_doc2vec, vectors_d2v, names_doc, steps=5, alpha=0.1, top=num_par_coll)
            search_result = blend_d2v_w2v(top_w2v, top_d2v, v=0.5, top=5)
        elif search_method == 'word2vec+inverted_index':
            top_w2v = search_w2v(query, model_without_pos, vectors_w2v, names_doc, tfidf, ind_text, multiply_tfidf=True, pos=False, top=num_par_coll)
            top_ind = get_search_result(query, top=num_text_coll)
            search_result = blend_w2v_index(top_w2v, top_ind, v=0.5, top=5)
        else:
            raise TypeError('unsupported search method')
    except:
        search_result = 'Неправильный запрос!'
    return search_result

In [209]:
search_method = 'word2vec+inverted_index'
query = 'Боится'

In [210]:
r = search(search_method, query, model_without_pos, model_doc2vec, df['w2v_tfidf'], df['d2v_hypo'], df['id_answer'], tfidf, 'del', len(df['id_answer']), len(data['description']), top=5)

In [211]:
r

[(1, 0.9201042619696476),
 (21, 0.5381342302685125),
 (7, 0.23127271141879036),
 (22, 0.16827498897532261),
 (27, 0.12391687330100565)]

In [202]:
data['description'][1]

'Щенки Пекинеса 2 кобеля. Дата рождения 11 мая 2018г.есть паспорт.привиты по возрасту. Воспитанные.росли в частном доме. На улицу просятся. Питание смешанное. Очень привязанные к друг другу. Поэтому желательно в одну семью. Прежде чем взять щенков, узнайте все о породе, осознаете что это члены семьи а не игрушка. Проверьте себя и всех ваших родственников в семье на аллергию!!! Щенки с отличными породными данными. Родители с родословной. Папа белый королевский Пекинес мама рыжая. (последние 2 фото) родители живут в одной семье со щенками'